<a href="https://colab.research.google.com/github/idan424/Comp.Learning_Neuro/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")


In [3]:
!git clone https://github.com/idan424/Comp.Learning_Neuro


Cloning into 'Comp.Learning_Neuro'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [117]:
PATH = '/content/Comp.Learning_Neuro/'
_files = ['Run_200 02.trc', 'Run_300 02.trc', 'Run_400 02.trc', 'Run_500 02.trc']
f = _files[0]

def load_data(f):
    _data = pd.read_table(PATH+f, header=4, index_col=0).iloc[:,:-1]
    _data = _data.rename(columns={'Unnamed: 1':'Time'})
    _data.reset_index(inplace=True)
    labels = pd.DataFrame(np.ones(len(_data)-2)*int(f[4:7]))
    return _data, labels

def add_av(_data):
    v_data = _data.copy().drop('Time', axis=1).diff()
    a_data = v_data.diff()

    v_data = v_data.rename(columns={n:f"V_{n}" for n in _data.columns})
    a_data = a_data.rename(columns={n:f"A_{n}" for n in _data.columns})

    return _data.join([v_data, a_data]).dropna()

def divide_dataset(_data, label):
    N = len(_data)//10
    return [ _data.loc[10*n:10*(n+1)-1] for n in range(N)], label[0:N]

def extract_features(seg_list):
    data = pd.DataFrame([segment_processing(seg) for seg in seg_list]) 
    return data

def segment_processing(_seg): #TODO: process a segment and concat results to the series
    seg = _seg.drop('Time', axis=1)

    mean_data = seg.mean().rename({n:f'mean_{n}'for n in seg.columns})
    std_data = seg.std().rename({n:f'std_{n}'for n in seg.columns})

    # פה צריך לחשוב על עוד פיצ'רים שאנחנו רוצים להכניס לשם

    return mean_data.append(std_data)  # returns a pd.Series()


def process_file(f):
    X,y = load_data(f)
    X = add_av(X)
    segments, y = divide_dataset(X, y)
    X = extract_features(segments)
    return (X,y)  # returns a tuple


files = _files
Xy = [process_file(f) for f in files]
Xy = list(zip(*Xy))  # this flip the tuple axis order

X, y = pd.concat(Xy[0]), pd.concat(Xy[1])



    mean_index      mean_X1      mean_Y1  ...  std_A_X42  std_A_Y42  std_A_Z42
0         14.5  1082.295521  1095.245126  ...   0.071294   0.424134   0.215719
1         23.5  1084.604082  1064.520578  ...   0.434725   0.788884   0.487597
2         33.5  1070.161569  1020.873275  ...   0.689687   0.725455   0.701105
3         43.5  1064.539822  1058.189897  ...   0.129670   0.510937   0.141166
4         53.5  1071.028948  1100.267987  ...   0.045978   0.134596   0.128669
..         ...          ...          ...  ...        ...        ...        ...
34       361.5   993.593984  1049.887616  ...   0.466979   0.480481   0.376438
35       371.5   973.351992  1018.116769  ...   0.413290   0.775528   0.726661
36       381.5   969.060577  1000.450789  ...   0.633939   0.577788   0.538884
37       391.5  1003.849861  1023.164068  ...   0.542608   0.228473   0.593324
38       401.5  1008.346280   995.546229  ...   0.695254   1.166001   0.287491

[190 rows x 762 columns]         0
0   200.0
1   20

## ##########################################################